In [35]:
import numpy as np
import tensorflow as tf

In [44]:
data = np.loadtxt('C:/Users/student/MachineLearning/csv/iris.csv', delimiter=',', dtype='float32')

np.random.shuffle(data)

train_data = data[:100]
test_data = data[100:]

x_data = train_data[:, 1:5]
y_data = train_data[:, :1]

print(train_data.shape, test_data.shape)


(100, 5) (50, 5)


In [45]:
def one_hot(y_data):
    #2차원 -> 1차원
    index = y_data.astype('uint8').flatten()
    
    rows = y_data.shape[0]
    cols = index.max() + 1
    temp = np.zeros((rows, cols), dtype = 'float32')
    temp[np.arange(rows), index] = 1.0
    return temp

y_data = one_hot(y_data)
#print(y_data)

num_input = x_data.shape[1] # 입력 노드 수
num_output = y_data.shape[1] # 출력 노드 수
num_hidden = 10

In [46]:
#신경망 모델 구성
X = tf.placeholder(tf.float32)
Y = tf.placeholder (tf.float32)

num_hidden2 = 20

W1 = tf.Variable(tf.random_uniform([num_input, num_hidden], -1, 1))
#b1 = tf.Variable(tf.zeros([num_hidden]))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([num_hidden, num_hidden2], -1, 1))
#b2 = tf.Variable(tf.zeros([num_output]))
L2 = tf.nn.relu(tf.matmul(L1, W2))

#편향 대체하는 가중치 추가
W3 = tf.Variable(tf.random_uniform([num_hidden2, num_output], -1, 1))
model = tf.matmul(L2, W3)

In [47]:
# 학습 회수 카운트
global_step = tf.Variable(0, trainable = False, name="global_step") 

In [48]:
#손실함수, 최적화 함수

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits = model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

In [52]:
with tf.Session() as sess:
    
    # 체크포인트가 존재하는지 검사
    ckpt = tf.train.get_checkpoint_state('./model_iris')
    saver = tf.train.Saver(tf.global_variables())
    
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        # 체크포인트가 존재하는 경우, 복원
        # global_step 값도 복원됨
        saver.restore(sess, ckpt.model_checkpoint_path)
    else :
        # 체크포인트가 존재하지 않는 경우, 전역 변수 초기화
        sess.run(tf.global_variables_initializer())

     # 학습
    feed_dict = {X: x_data, Y: y_data}
    for step in range(2):
        sess.run(train_op, feed_dict = feed_dict)
        print('Step: {}, Cost: {:.3f}'.format(
            sess.run(global_step),
            sess.run(cost, feed_dict=feed_dict)))
    
    # 체크포인트 저장
    saver.save(sess, './model_iris/dnn.ckpt', global_step=global_step)
   
    #학습 결과 확인
    prediction = tf.argmax(model, axis=1)
    target = tf.argmax(Y, axis=1)
    print("예측값 : ", sess.run(prediction, feed_dict={X : x_data}))
    print("실제값 : ", sess.run(target, feed_dict={Y : y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    print('정확도 : {:.2f}'.format(sess.run(accuracy* 100,feed_dict={ X: x_data,Y: y_data})) )

INFO:tensorflow:Restoring parameters from ./model_iris\dnn.ckpt-6
Step: 7, Cost: 3.780
Step: 8, Cost: 3.546
예측값 :  [1 1 1 2 2 2 1 2 1 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 1 1 2 2 2 2 2 2 2 2 1
 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 1 2 2 1 2 2 2 1 2 2 1 2 2 2 1
 2 1 2 1 1 1 2 1 2 2 1 2 1 2 1 2 2 1 2 2 1 1 2 2 2 2]
실제값 :  [0 0 0 1 2 2 0 1 0 1 0 2 1 1 0 1 2 1 1 1 2 1 1 2 0 2 0 0 1 1 1 1 1 2 2 1 0
 2 0 2 1 1 2 0 2 2 2 0 1 2 1 2 2 1 2 1 2 0 2 0 2 0 0 2 1 2 0 1 2 0 2 2 1 0
 1 0 1 0 0 0 2 0 1 2 0 1 0 1 0 2 1 0 1 1 0 0 2 1 2 1]
정확도 : 32.00
